# Procédure de la récupération des données Météo France SWI 1969—2022, mailles et métadonnées

## I. Télécharger les données

1. Se rendre à la [page dédiée][1]:
   * télécharger l'archive zip avec toutes les mailles depuis la rubrique `Téléchargement` ;
   * télécharger le fichier `Coordonnées des mailles` depuis la rubrique `Documentation`.

## II. Souder les fichiers

3. Décompresser l'archive téléchargée.
4. Extraire la ligne d'en-tête de l'un des fichiers CSV et la sauvegarder dans un fichier à part (`swi_mailles_header`).
5. Souder le contenu de tous les fichiers CSV à l'exception de la première ligne de chacun en l'enregistrant dans un fichier (`swi_mailles_headless`).
6. Souder ensemble `swi_mailles_header` et `swi_mailles_headless` (`swi_mailles`).
7. Effectuer la jointure entre le fichier mailles et celui des métadonnées.

[1]: https://donneespubliques.meteofrance.fr/?fond=produit&id_produit=301&id_rubrique=40

## III. Effectuer la jointure entre les fichiers `swi_mailles.csv` et `metadonnees_swi_276.csv`

### 1. Charger les données du fichier `swi_mailles.csv`

En prenant bien soin d'indiquer la virgule en tant que séparateur décimal.

In [43]:
import pandas as pd

df_mailles = pd.read_csv('swi_mailles.csv', sep=';', decimal=',')
df_mailles

C:\Users\Administrateur\AppData\Local\Temp\ipykernel_6916\530456940.py:3: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mailles = pd.read_csv('swi_mailles.csv', sep=';', decimal=',')


,NUMERO,LAMBX,LAMBY,DATE,SWI_UNIF_MENS3
0,2,641374,7106309,196901,1.013
1,2,641374,7106309,196902,1.054
2,2,641374,7106309,196903,1.089
3,2,641374,7106309,196904,1.062
4,2,641374,7106309,196905,1.054
...,...,...,...,...,...
5819722,9892,1215772,6046242,202208,0.007
5819723,9892,1215772,6046242,202209,-0.02
5819724,9892,1215772,6046242,202210,0.134
5819725,9892,1215772,6046242,202211,0.431


### 2. Charger les données du fichier `metadonnees_swi_276.csv`

D'après l'éditeur des données :
* `num_maille` : numéro de la maille
* `lambx`, `lamby` : coordonnées de la maille en Lambert 2 étendu, en hectomètres
* `lambx93`, `lamby93` : coordonnées de la maille en Lambert 93
* `lat_dg`, `lon_dg` : coordonnées de la maille en degrés décimaux

In [44]:
import pandas as pd

df_metadata = pd.read_csv('metadonnees_swi_276.csv', sep=';')
df_metadata

,num_maille,lambx,lamby,lambx93,lamby93,lat_dg,lon_dg
0,2,5880,26730,641374,7106309,51.0497,2.16654
1,3,5960,26730,649370,7106242,51.0498,2.28031
2,4,6040,26730,657366,7106175,51.0498,2.39409
3,5,6120,26730,665362,7106108,51.0497,2.50786
4,6,6200,26730,673358,7106041,51.0495,2.62163
...,...,...,...,...,...,...,...
8976,9888,11800,16330,1223902,6062151,41.4739,9.26140
8977,9889,11640,16250,1207852,6054302,41.4148,9.06318
8978,9890,11720,16250,1215842,6054231,41.4087,9.15811
8979,9891,11800,16250,1223832,6054161,41.4024,9.25302


### 3. Effectuer la jointure entre les fichiers `swi_mailles.csv` et `metadonnees_swi_276.csv`

In [45]:
df_mailles_with_metadata = pd.merge(df_mailles, df_metadata, how='left', left_on=['NUMERO', 'LAMBX', 'LAMBY'], right_on=['num_maille', 'lambx93', 'lamby93']).convert_dtypes(convert_integer=True)
df_mailles_with_metadata

,NUMERO,LAMBX,LAMBY,DATE,SWI_UNIF_MENS3,num_maille,lambx,lamby,lambx93,lamby93,lat_dg,lon_dg
0,2,641374,7106309,196901,1.013,2,5880,26730,641374,7106309,51.0497,2.16654
1,2,641374,7106309,196902,1.054,2,5880,26730,641374,7106309,51.0497,2.16654
2,2,641374,7106309,196903,1.089,2,5880,26730,641374,7106309,51.0497,2.16654
3,2,641374,7106309,196904,1.062,2,5880,26730,641374,7106309,51.0497,2.16654
4,2,641374,7106309,196905,1.054,2,5880,26730,641374,7106309,51.0497,2.16654
...,...,...,...,...,...,...,...,...,...,...,...,...
5819722,9892,1215772,6046242,202208,0.007,9892,11720,16170,1215772,6046242,41.3372,9.14986
5819723,9892,1215772,6046242,202209,-0.02,9892,11720,16170,1215772,6046242,41.3372,9.14986
5819724,9892,1215772,6046242,202210,0.134,9892,11720,16170,1215772,6046242,41.3372,9.14986
5819725,9892,1215772,6046242,202211,0.431,9892,11720,16170,1215772,6046242,41.3372,9.14986


### 4. Supprimer les champs en doublon

In [46]:
df_mailles_with_metadata = df_mailles_with_metadata.drop(labels=['NUMERO', 'LAMBX', 'LAMBY'], axis='columns')
df_mailles_with_metadata

,DATE,SWI_UNIF_MENS3,num_maille,lambx,lamby,lambx93,lamby93,lat_dg,lon_dg
0,196901,1.013,2,5880,26730,641374,7106309,51.0497,2.16654
1,196902,1.054,2,5880,26730,641374,7106309,51.0497,2.16654
2,196903,1.089,2,5880,26730,641374,7106309,51.0497,2.16654
3,196904,1.062,2,5880,26730,641374,7106309,51.0497,2.16654
4,196905,1.054,2,5880,26730,641374,7106309,51.0497,2.16654
...,...,...,...,...,...,...,...,...,...
5819722,202208,0.007,9892,11720,16170,1215772,6046242,41.3372,9.14986
5819723,202209,-0.02,9892,11720,16170,1215772,6046242,41.3372,9.14986
5819724,202210,0.134,9892,11720,16170,1215772,6046242,41.3372,9.14986
5819725,202211,0.431,9892,11720,16170,1215772,6046242,41.3372,9.14986


### 5. Convertir les noms des champs en minuscule

In [47]:
print("Noms des champs d'origine :", df_mailles_with_metadata.columns.to_list())
df_mailles_with_metadata.columns = [n.lower() for n in df_mailles_with_metadata.columns.to_list()]
print("Noms des champs renommés :")
df_mailles_with_metadata

Noms des champs d'origine : ['DATE', 'SWI_UNIF_MENS3', 'num_maille', 'lambx', 'lamby', 'lambx93', 'lamby93', 'lat_dg', 'lon_dg']
Noms des champs renommés :


,date,swi_unif_mens3,num_maille,lambx,lamby,lambx93,lamby93,lat_dg,lon_dg
0,196901,1.013,2,5880,26730,641374,7106309,51.0497,2.16654
1,196902,1.054,2,5880,26730,641374,7106309,51.0497,2.16654
2,196903,1.089,2,5880,26730,641374,7106309,51.0497,2.16654
3,196904,1.062,2,5880,26730,641374,7106309,51.0497,2.16654
4,196905,1.054,2,5880,26730,641374,7106309,51.0497,2.16654
...,...,...,...,...,...,...,...,...,...
5819722,202208,0.007,9892,11720,16170,1215772,6046242,41.3372,9.14986
5819723,202209,-0.02,9892,11720,16170,1215772,6046242,41.3372,9.14986
5819724,202210,0.134,9892,11720,16170,1215772,6046242,41.3372,9.14986
5819725,202211,0.431,9892,11720,16170,1215772,6046242,41.3372,9.14986


### 6. Sauvegarder le `DataFrame` dans un nouveau fichier CSV

Sans garder le champ `index`

In [49]:
df_mailles_with_metadata.to_csv('swi_mailles_with_metadata.csv', index=False)